<a href="https://colab.research.google.com/github/Naveensadanandan/Pytorch/blob/main/06_Transformer_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers Use Cases

## Using pipeline() function from Transformers

### The most basic object in the 🤗 Transformers library is the pipeline() function. It connects a model with its necessary preprocessing and postprocessing steps, allowing us to directly input any text and get an intelligible answer:

There are three main steps involved when you pass some text to a pipeline:

1.The text is preprocessed into a format the model can understand.(This pipeline selects a particular pretrained model that has been fine-tuned for sentiment analysis in English.

2.The preprocessed inputs are passed to the model.

3.The predictions of the model are post-processed, so you can make sense of them.

### 1.Sentiment analysis pipeline

In [32]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
text = "Today is so peacefull"
classifier(text)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[{'label': 'POSITIVE', 'score': 0.9985666871070862}]

### 2.feature-extraction pipeline

In [33]:
classifier = pipeline("feature-extraction")
feature = classifier(text)

No model was supplied, defaulted to distilbert/distilbert-base-cased and revision 935ac13 (https://huggingface.co/distilbert/distilbert-base-cased).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

### Zero shot classification

In [34]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification")
classifier(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business"],
)


No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

{'sequence': 'This is a course about the Transformers library',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.8445994257926941, 0.11197388917207718, 0.0434267558157444]}

'''
This pipeline is called zero-shot because you don’t need to fine-tune the model on your data to use it. It can directly return probability scores for any list of labels you want!
'''

# How do Transformers work
### All the Transformer models like (GPT, BERT, BART, T5, etc.) have been trained as language models. This means they have been trained on large amounts of raw text in a self-supervised fashion. Self-supervised learning is a type of training in which the objective is automatically computed from the inputs of the model. That means that humans are not needed to label the data!

## Transfer Learning
For example, one could leverage a pretrained model trained on the English language and then fine-tune it on an arXiv corpus, resulting in a science/research-based model. The fine-tuning will only require a limited amount of data: the knowledge the pretrained model has acquired is “transferred,” hence the term transfer learning.

# Using Transformer models

In [71]:
sentences = [
    "The customer service at the bank was excellent and very helpful.",
    "I'm extremely disappointed with the high fees charged for the transaction.",
    "The new mobile banking app is very user-friendly and easy to navigate.",
    "I had a terrible experience with their support team; they never responded to my query.",
    "I’m thrilled with how fast the loan approval process was!",
    "The website keeps crashing, and it’s very frustrating to complete any transaction.",
    "I'm satisfied with the interest rates they offer for savings accounts.",
    "This is the worst financial service I have ever used.",
    "The security features on their platform make me feel safe when transferring money.",
    "I am unhappy with the constant delays in processing payments."
]

## Tokenizing Raw sentences

In [72]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [73]:
input_ids = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

In [74]:
input_ids["input_ids"].shape

torch.Size([10, 19])

## Passing input_ids to distilbert model

In [75]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [76]:
logits = model(**input_ids)
print(logits.logits.shape)

torch.Size([10, 2])


## Using softmax and argmax for prediction

In [77]:
import torch
probs = torch.softmax(logits.logits, dim=1)
predictions = torch.argmax(probs, dim=1)
print(predictions)

tensor([1, 0, 1, 0, 1, 0, 1, 0, 1, 0])


In [83]:
probs

tensor([[1.6071e-04, 9.9984e-01],
        [9.9979e-01, 2.0941e-04],
        [2.2732e-03, 9.9773e-01],
        [9.9970e-01, 3.0072e-04],
        [1.2419e-03, 9.9876e-01],
        [9.9951e-01, 4.9429e-04],
        [9.6940e-04, 9.9903e-01],
        [9.9979e-01, 2.0506e-04],
        [1.9197e-02, 9.8080e-01],
        [9.9978e-01, 2.2358e-04]], grad_fn=<SoftmaxBackward0>)

In [84]:
import pandas as pd
df =pd.DataFrame({"Sentences" : sentences, "Sentiment": predictions})
df

,Sentences,Sentiment
0,The customer service at the bank was excellent...,1
1,I'm extremely disappointed with the high fees ...,0
2,The new mobile banking app is very user-friend...,1
3,I had a terrible experience with their support...,0
4,I’m thrilled with how fast the loan approval p...,1
5,"The website keeps crashing, and it’s very frus...",0
6,I'm satisfied with the interest rates they off...,1
7,This is the worst financial service I have eve...,0
8,The security features on their platform make m...,1
9,I am unhappy with the constant delays in proce...,0


In [79]:
model_path = "model_path"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('model_path/tokenizer_config.json',
 'model_path/special_tokens_map.json',
 'model_path/vocab.txt',
 'model_path/added_tokens.json',
 'model_path/tokenizer.json')

## Loading saved model

In [80]:
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer

config = AutoConfig.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, config = config)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [81]:
from transformers import PreTrainedModel, PreTrainedTokenizer
def make_predictions(text : str,
                     model : PreTrainedModel,
                     tokenizer : PreTrainedTokenizer) -> int:
                     input_ids = tokenizer(text, return_tensors="pt")
                     output = model(**input_ids)
                     prediction = torch.softmax(output.logits, dim=1)
                     return prediction

In [82]:
text = "The customer service at the bank was excellent and very helpful."
make_predictions(text, model, tokenizer)

tensor([[1.6071e-04, 9.9984e-01]], grad_fn=<SoftmaxBackward0>)